In [1]:
!ls *.sh

macav2livneh_wget.sh


In [2]:
with open('macav2livneh_wget.sh') as fo:
    lines = fo.readlines()
    
lines

['#!/bin/bash \n',
 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc" -O macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc  \n',
 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily.nc" -O macav2livneh_huss_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily.nc  \n',
 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily.nc" -O macav2livneh_huss_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily.nc  \n',
 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh

In [3]:
lines[1]

'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc" -O macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc  \n'

In [4]:
words = lines[1].split()
for w in words:
    if 'http' in w:
        print(w)
        http_line = w
http_root=http_line.split('macav2livneh/')[0][1:]

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc"


In [5]:
lines.pop(0)

lines[0]

'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/macav2livneh/CCSM4/macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc" -O macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc  \n'

In [6]:
outs=[]
for line in lines:
    if '-O' in line:
        out=line.split('-O')[1][1:]
        out = out.rstrip()
        out = out.split('.nc')[0]
        print(out)
        outs.append(out)

macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_rcp85_2006_2025_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_rcp85_2026_2045_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_rcp85_2046_2065_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_rcp85_2066_2085_CONUS_daily
macav2livneh_huss_CCSM4_r6i1p1_rcp85_2086_2099_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_rcp85_2006_2025_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_rcp85_2026_2045_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_rcp85_2046_2065_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_rcp85_2066_2085_CONUS_daily
macav2livneh_pr_CCSM4_r6i1p1_rcp85_2086_2099_CONUS_daily
macav2livneh_rsds_CCSM4_r6i1p1_historical_

In [7]:
outs

['macav2livneh_huss_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_rcp85_2006_2025_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_rcp85_2026_2045_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_rcp85_2046_2065_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_rcp85_2066_2085_CONUS_daily',
 'macav2livneh_huss_CCSM4_r6i1p1_rcp85_2086_2099_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_historical_1970_1989_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_historical_1990_2005_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_rcp85_2006_2025_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_rcp85_2026_2045_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_rcp85_2046_2065_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_rcp85_2066_2085_CONUS_daily',
 'macav2livneh_pr_CCSM4_r6i1p1_rcp85_20

In [8]:
dfis=[]
for out in outs:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

In [9]:
dfis

[{'dataset': 'macav2livneh',
  'measure': 'huss',
  'model': 'CCSM4',
  'r6': 'r6i1p1',
  'epoch': 'historical',
  'start_year': '1950',
  'end_year': '1969',
  'region': 'CONUS',
  'frequency': 'daily'},
 {'dataset': 'macav2livneh',
  'measure': 'huss',
  'model': 'CCSM4',
  'r6': 'r6i1p1',
  'epoch': 'historical',
  'start_year': '1970',
  'end_year': '1989',
  'region': 'CONUS',
  'frequency': 'daily'},
 {'dataset': 'macav2livneh',
  'measure': 'huss',
  'model': 'CCSM4',
  'r6': 'r6i1p1',
  'epoch': 'historical',
  'start_year': '1990',
  'end_year': '2005',
  'region': 'CONUS',
  'frequency': 'daily'},
 {'dataset': 'macav2livneh',
  'measure': 'huss',
  'model': 'CCSM4',
  'r6': 'r6i1p1',
  'epoch': 'rcp85',
  'start_year': '2006',
  'end_year': '2025',
  'region': 'CONUS',
  'frequency': 'daily'},
 {'dataset': 'macav2livneh',
  'measure': 'huss',
  'model': 'CCSM4',
  'r6': 'r6i1p1',
  'epoch': 'rcp85',
  'start_year': '2026',
  'end_year': '2045',
  'region': 'CONUS',
  'frequen

In [10]:
import pandas as pd  
df = pd.DataFrame(dfis)

In [11]:
df

,dataset,measure,model,r6,epoch,start_year,end_year,region,frequency
0,macav2livneh,huss,CCSM4,r6i1p1,historical,1950,1969,CONUS,daily
1,macav2livneh,huss,CCSM4,r6i1p1,historical,1970,1989,CONUS,daily
2,macav2livneh,huss,CCSM4,r6i1p1,historical,1990,2005,CONUS,daily
3,macav2livneh,huss,CCSM4,r6i1p1,rcp85,2006,2025,CONUS,daily
4,macav2livneh,huss,CCSM4,r6i1p1,rcp85,2026,2045,CONUS,daily
5,macav2livneh,huss,CCSM4,r6i1p1,rcp85,2046,2065,CONUS,daily
6,macav2livneh,huss,CCSM4,r6i1p1,rcp85,2066,2085,CONUS,daily
7,macav2livneh,huss,CCSM4,r6i1p1,rcp85,2086,2099,CONUS,daily
8,macav2livneh,pr,CCSM4,r6i1p1,historical,1950,1969,CONUS,daily
9,macav2livneh,pr,CCSM4,r6i1p1,historical,1970,1989,CONUS,daily


In [12]:
pr_df = df[df['measure'] == 'pr']

In [13]:
pr_df

,dataset,measure,model,r6,epoch,start_year,end_year,region,frequency
8,macav2livneh,pr,CCSM4,r6i1p1,historical,1950,1969,CONUS,daily
9,macav2livneh,pr,CCSM4,r6i1p1,historical,1970,1989,CONUS,daily
10,macav2livneh,pr,CCSM4,r6i1p1,historical,1990,2005,CONUS,daily
11,macav2livneh,pr,CCSM4,r6i1p1,rcp85,2006,2025,CONUS,daily
12,macav2livneh,pr,CCSM4,r6i1p1,rcp85,2026,2045,CONUS,daily
13,macav2livneh,pr,CCSM4,r6i1p1,rcp85,2046,2065,CONUS,daily
14,macav2livneh,pr,CCSM4,r6i1p1,rcp85,2066,2085,CONUS,daily
15,macav2livneh,pr,CCSM4,r6i1p1,rcp85,2086,2099,CONUS,daily


In [14]:
!mkdir -p ~/junkbox

In [15]:
http_root

'http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/'

In [16]:
myl = list(pr_df.loc[8])

In [17]:
myl

['macav2livneh',
 'pr',
 'CCSM4',
 'r6i1p1',
 'historical',
 '1950',
 '1969',
 'CONUS',
 'daily']

In [18]:
crafted_file_name = '_'.join(myl) + '.nc'

In [19]:
crafted_file_name

'macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc'

In [20]:
myd = dict(pr_df.loc[8])

In [21]:
myd

{'dataset': 'macav2livneh',
 'measure': 'pr',
 'model': 'CCSM4',
 'r6': 'r6i1p1',
 'epoch': 'historical',
 'start_year': '1950',
 'end_year': '1969',
 'region': 'CONUS',
 'frequency': 'daily'}

In [22]:
from_dir = myd['dataset'] + '/' + myd['model']

from_dir

'macav2livneh/CCSM4'

In [23]:
import os

home = os.getenv('HOME')
home

'/home/jupyter-rouze'

In [24]:
to_dir= home + '/junkbox/' + myd['measure']

to_dir

'/home/jupyter-rouze/junkbox/pr'

In [25]:
try:
    os.mkdir(to_dir);
except:
    pass

In [26]:
wget_cmd = http_root + '/' + from_dir + '/' + crafted_file_name + ' -O ' + to_dir + '/' + crafted_file_name 

In [27]:
wget_cmd

'http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE//macav2livneh/CCSM4/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc -O /home/jupyter-rouze/junkbox/pr/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc'

In [28]:
wget = 'wget -nc -c -nd ' 

In [29]:
wget_cmd = wget + wget_cmd

wget_cmd

'wget -nc -c -nd http://thredds.northwestknowledge.net:8080/thredds/fileServer/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE//macav2livneh/CCSM4/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc -O /home/jupyter-rouze/junkbox/pr/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc'

In [30]:
%%time
os.system(wget_cmd)

CPU times: user 11.6 ms, sys: 14.5 ms, total: 26.1 ms
Wall time: 5min 48s


0

In [31]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(to_dir) if isfile(join(to_dir, f))]

onlyfiles

['macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc']

# Move it to the cloud 

In [32]:
import boto3
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)


In [33]:
bucket = 'dev-et-data'

local_file = to_dir + '/' + crafted_file_name

bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name

bucket_filepath

'raw/pr/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc'

In [34]:
%%time
s3_push_delete_local(local_file, bucket, bucket_filepath)

dev-et-data raw/pr/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc
CPU times: user 34.8 s, sys: 49.7 s, total: 1min 24s
Wall time: 45.5 s


In [35]:
import fsspec


In [36]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [37]:
file_objects = fs.ls('dev-et-data')
file_objects

['dev-et-data/',
 'dev-et-data/NA_data_for_cloud',
 'dev-et-data/NDVI_filled',
 'dev-et-data/Readme-bucket.md',
 'dev-et-data/in',
 'dev-et-data/out_drb_20years',
 'dev-et-data/raw',
 'dev-et-data/test',
 'dev-et-data/tiles',
 'dev-et-data/tony.txt',
 'dev-et-data/zarr']

In [39]:
file_objects = fs.ls('dev-et-data/raw/pr/')
file_objects

['dev-et-data/raw/pr/macav2livneh_pr_CCSM4_r6i1p1_historical_1950_1969_CONUS_daily.nc']